In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '../exps/')
import posgres_helper as db_helper
from retrieving_botscore import *

# Check botscore of friends/followers of bots

In [ ]:
users_to_check_botscore = db_helper.getDataframeFromQuery(
    db_helper.connect_db(), 
    """
    select 
        c.t_usr_id_ego, 
        c.t_usr_id_conn, 
        c.conn_type
    from
        bot b,
        connections c
    where b.twitter_user_id = c.t_usr_id_ego
    ;"""
)
users_to_check_botscore.drop_duplicates(inplace=True)

In [ ]:
twitter_id_alignment_dict = {
    "<BOT_1_ID>": "Left",
    "<BOT_2_ID>": "Left",
    # ...
    "<BOT_4_ID>": "C.Left",
    "<BOT_5_ID>": "C.Left",
    # ...
    "<BOT_n+1_ID>": "Right",
    "<BOT_n+2_ID>": "Right",
}

In [ ]:
users_to_check_botscore["seed"] = users_to_check_botscore.t_usr_id_ego.apply(
    twitter_id_alignment_dict.get
)

users_to_check_botscore.conn_type = users_to_check_botscore.conn_type.apply(
    lambda x: "follower" if x else "friend"
)

## saving bot scores to json files for later processing

In [ ]:
tw_keys = {
    "consumer_key": "<replace_here>",
    "consumer_secret": "<replace_here>",
    "access_token": "<replace_here>",
    "access_token_secret": "<replace_here>",
    "mashape_key": "<replace_here>",
}
check_botscore = RetrieveBotscore(tw_keys)
check_botscore.check_botometer_scores(users_to_check_botscore.t_usr_id_conn.unique())

## Reading bot scores from json file

In [ ]:
json_file = "<FILL_HERE_THE_NAME_OF_THE_FILE_SAVED_IN_THE_PREVIOUS_STEP>"
botscore_measurements = check_botscore.loadJson(json_file)[[
    "user_id_str","user_screen_name","scores_english"
]]
botscore_measurements = botscore_measurements.join(
    pd.DataFrame(users_to_check_botscore,columns=["user_id_str"]).set_index("user_id_str"),
    on="user_id_str",
    how="outer"
)
botscore_measurements = botscore_measurements[~botscore_measurements.user_id_str.isna()]
botscore_measurements.set_index("user_id_str", inplace=True)
botscore_dict = botscore_measurements.scores_english.to_dict()

connections_overall_botscores = users_to_check_botscore.groupby(
    ["conn_type","seed"]
).apply(
    lambda x: pd.Series([
        botscore_dict.get(str(user_id)) 
        for user_id in pd.Series(np.concatenate(x.t_usr_id_conn.values)).unique()
    ]).dropna().to_list()
)

connections_overall_botscores.to_csv("../data/connections_seeds_unique_botscores.csv")